In [2]:
from typing import List, Dict
import requests

In [ ]:


def get_top_holders(token_address: str, api_key: str = None) -> List[Dict]:
    if api_key:
        url = f"https://api.etherscan.io/api?module=token&action=tokenholders&address={token_address}&page=1&offset=100&apikey={api_key}"
        response = requests.get(url)
        return response.json()['result']
    else:
        return mock_top_holders()  # Returns example data

In [30]:
import requests

url = "https://api.coingecko.com/api/v3/asset_platforms"

headers = {
    "accept": "application/json",
    "x-cg-api-key": "CG-PTSfJuf6dbcdXusfkcFRTrLV"
}

response = requests.get(url, headers=headers)

response.json()

[{'id': 'valobit',
  'chain_identifier': None,
  'name': 'Valobit',
  'shortname': '',
  'native_coin_id': 'valobit',
  'image': {'thumb': 'https://coin-images.coingecko.com/asset_platforms/images/182/thumb/valobit.png?1708317270',
   'small': 'https://coin-images.coingecko.com/asset_platforms/images/182/small/valobit.png?1708317270',
   'large': 'https://coin-images.coingecko.com/asset_platforms/images/182/large/valobit.png?1708317270'}},
 {'id': 'arbitrum-nova',
  'chain_identifier': 42170,
  'name': 'Arbitrum Nova',
  'shortname': '',
  'native_coin_id': 'ethereum',
  'image': {'thumb': 'https://coin-images.coingecko.com/asset_platforms/images/93/thumb/AN_logomark.png?1706606703',
   'small': 'https://coin-images.coingecko.com/asset_platforms/images/93/small/AN_logomark.png?1706606703',
   'large': 'https://coin-images.coingecko.com/asset_platforms/images/93/large/AN_logomark.png?1706606703'}},
 {'id': 'factom',
  'chain_identifier': None,
  'name': 'Factom',
  'shortname': '',
  'n

In [88]:
def calculate_price_impact(pool_address, token_in, amount_in, fee):
    """
    Calculate price impact for a swap in a PancakeSwap V3 pool
    Args:
        pool_address: Pool smart contract address
        token_in: Token smart contract address
        amount_in: Amount of token to swap
    Returns:
        Price impact
    """

    pool_data = get_pool_data(pool_address)
    if not pool_data:
        return
    
    inital_reserve_0 = float(pool_data['tvlToken0'])
    inital_reserve_1 = float(pool_data['tvlToken1'])

    constant_product = inital_reserve_0 * inital_reserve_1

    # Determine token in/out and adjust amounts accordingly
    if token_in == pool_data['token0']['id']:  # WBNB in this case
        # Swapping token0 (WBNB) for token1 (BR)
        token_in_symbol = pool_data['token1']['symbol']
        initial_price = inital_reserve_0 / inital_reserve_1  # BR per WBNB
        new_reserve_0 = inital_reserve_0 + amount_in * (1 - fee)
        new_reserve_1 = constant_product / new_reserve_0
        amount_out = inital_reserve_1 - new_reserve_1
        trade_price = (amount_in * (1 - fee))  / amount_out
        price_impact = 1 - initial_price / trade_price
    elif token_in == pool_data['token1']['id']:
        # Swapping token1 (BR) for token0 (WBNB)
        token_in_symbol = pool_data['token1']['symbol']
        initial_price = inital_reserve_1 / inital_reserve_0  # WBNB per BR
        new_reserve_1 = inital_reserve_1 + amount_in * (1 - fee)
        new_reserve_0 = constant_product / new_reserve_1
        amount_out = inital_reserve_0 - new_reserve_0
        trade_price = (amount_in * (1 - fee)) / amount_out
        price_impact = 1 - initial_price / trade_price
    else:
        print('Please check pool and token addresses')
        return

    print(f"Pool: {pool_data['token0']['symbol']}/{pool_data['token1']['symbol']}")
    print(f"Current Price: 1 {pool_data['token0']['symbol']} = {pool_data['token0Price']} {pool_data['token1']['symbol']}")
    print(f"TVL: ${float(pool_data['tvlUSD']):,.2f}")
    print(f"\nPrice impact for swapping {amount_in} {token_in_symbol}: {price_impact * 100:.6f}%")

    return price_impact

In [99]:
token_address = '0xff7d6a96ae471bbcd7713af9cb1feeb16cf56b41'
pool_address = '0xf95f84e2bad9c234f93dd66614b82f9a854b452e'
supply_trade_pct = 0.005

token_supply = get_token_data('0xff7d6a96ae471bbcd7713af9cb1feeb16cf56b41')['supply']
calculate_price_impact(pool_address,token_address,token_supply * supply_trade_pct,0.005)

Pool: WBNB/BR
Current Price: 1 WBNB = 0.00008707411741298737 BR
TVL: $2,111,825.07

Price impact for swapping 1050185.8849813608 BR: 3.803078%


0.03803078290902662

In [ ]:
from pancake_simulation import PancakeSwapAnalyzer

# Example usage
analyzer = PancakeSwapAnalyzer(coingecko_api_key="CG-PTSfJuf6dbcdXusfkcFRTrLV")
pool_address = "0xf95f84e2bad9c234f93dd66614b82f9a854b452e"
token_in = "0xff7d6a96ae471bbcd7713af9cb1feeb16cf56b41"
token_supply = analyzer.get_token_data(token_in)['supply']
supply_trade_pct = 0.005
fee = 0.005  # 0.5% fee  

price_impact = analyzer.calculate_price_impact(pool_address, token_in, token_supply * supply_trade_pct, fee)

Pool: WBNB/BR
Current Price: 1 WBNB = 0.0000870609380511369 BR
TVL: $2,113,080.87

Price impact for swapping 1049811.3283494036 BR: 3.808569%
